[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-research/imagenet-mistakes/blob/main/notebooks/When_does_dough_become_a_bagel__ImageNet_M_Evaluation.ipynb)

This notebook demonstrate both ImageNet-M evaluation as well as 
Multi-Label validation and standard imagenet validation using pre-computed
logits for the ViT3B and Greedy Soups models.


In [ ]:
#@title Run some imports.
import json
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

In [ ]:
#@title Accuracy and Loader functions.
def imagenet_val_accuracy(imagenet_val_ds, logits_dict):
  correct = 0
  for example in tqdm.tqdm(imagenet_val_ds):
    gt_label = example['label']
    file_name = example['file_name'].numpy()
    logits = logits_dict[file_name]
    scores = tf.nn.softmax(logits).numpy()
    if tf.nn.in_top_k(np.reshape(gt_label, [1]), scores[np.newaxis], k=1).numpy()[0]:
      correct += 1
  return correct / float(len(imagenet_val_ds))

def imagenet_multilabel_accuracy(imagenet_multilabel_ds, logits_dict):
  num_correct_per_class = {}
  num_images_per_class = {}
  for example in imagenet_multilabel_ds:
    # We ignore all problematic images
    if example['is_problematic'].numpy():
        continue

    correct_labels = (list(example['correct_multi_labels']) +
                      list(example['unclear_multi_labels']))

    # The label of the image in ImageNet
    cur_class = example['original_label'].numpy()

    # If we haven't processed this class yet, set the counters to 0
    if cur_class not in num_correct_per_class:
      num_correct_per_class[cur_class] = 0
      assert cur_class not in num_images_per_class
      num_images_per_class[cur_class] = 0

    num_images_per_class[cur_class] += 1

    # Get the predictions for this image
    file_name = example['file_name'].numpy()
    logits = logits_dict[file_name]
    scores = tf.nn.softmax(logits).numpy()
    top_pred = np.argmax(scores)

    # We count a prediction as correct if it is marked as correct or unclear
    # (i.e., we are lenient with the unclear labels)
    if top_pred in correct_labels:
      num_correct_per_class[cur_class] += 1

  # Check that we have collected accuracy data for each of the 1,000 classes
  num_classes = 1000
  assert len(num_correct_per_class) == num_classes
  assert len(num_images_per_class) == num_classes

  # Compute the per-class accuracies and then average them
  final_avg = 0
  for cid in range(num_classes):
    assert cid in num_correct_per_class
    assert cid in num_images_per_class
    final_avg += num_correct_per_class[cid] / num_images_per_class[cid]
  final_avg /= num_classes
  return final_avg

def imagenet_m_correct(imagenet_m_ds, logits_dict):
  correct = 0
  for example in imagenet_m_ds:
    correct_labels = (list(example['correct_multi_labels']) +
                      list(example['unclear_multi_labels']))
    file_name = example['file_name'].numpy()
    logits = logits_dict[file_name]
    scores = tf.nn.softmax(logits).numpy()
    top_pred = np.argmax(scores)
    if top_pred in correct_labels:
      correct += 1
  return correct

def load_npy_as_dict(path):
  logits_array = np.load(tf.io.gfile.GFile(path, 'rb'))
  ret = {}
  for i in range(len(logits_array)):
    filename = f'ILSVRC2012_val_{i+1:08}.JPEG'.encode()
    ret[filename] = logits_array[i]
  return ret

In [ ]:
#@title Download all relevant assets.
!wget https://raw.githubusercontent.com/modestyachts/ImageNetV2/master/data/metadata/class_info.json
!wget https://raw.githubusercontent.com/tensorflow/datasets/master/tensorflow_datasets/image_classification/imagenet2012_validation_labels.txt
!wget https://storage.googleapis.com/brain-car-datasets/imagenet-mistakes/human_accuracy_v3.0.0.json
!wget https://storage.googleapis.com/brain-car-datasets/imagenet-mistakes/logits/vit3b.npz
!wget https://storage.googleapis.com/brain-car-datasets/imagenet-mistakes/logits/greedysoups.npz


Now we load and preprocess the files downloaded.  It should take less than a minute to run.

In [ ]:
# WNID to CID mapping.
imagenet_class_info = json.load(open('class_info.json'))
wnid_to_cid = {}
for entry in imagenet_class_info:
  wnid_to_cid[entry['wnid']] = entry['cid']

# Validation set list, in wnid form.
labels_list = open('imagenet2012_validation_labels.txt').read().strip().splitlines()

# Human accuracy json
human_accuracy = json.load(open('human_accuracy_v3.0.0.json'))
imagenetv1_prefix = 'ILSVRC2012_val_'
prefix_len = len(imagenetv1_prefix)
annotated_images = {}
for image_name, data in human_accuracy['initial_annots'].items():
  if image_name.startswith(imagenetv1_prefix):
    annotated_images[image_name] = data

problematic_images = set(human_accuracy['problematic_images'].keys())
imagenet_m_2022_files = set(human_accuracy['imagenet_m'])

# Set up data structure of inputs.
imagenet_multilabel_v3 = {}
imagenet_m_2022 = {}
for image_name, data in annotated_images.items():
  is_problematic = image_name in problematic_images
  correct_multi_labels = data.get('correct', [])
  wrong_multi_labels = data.get('wrong', [])
  unclear_multi_labels = data.get('unclear', [])

  correct_cids = [wnid_to_cid[x] for x in correct_multi_labels]
  wrong_cids = [wnid_to_cid[x] for x in wrong_multi_labels]
  unclear_cids = [wnid_to_cid[x] for x in unclear_multi_labels]

  image_id = int(image_name.split('_')[-1][:-5])
  original_label = wnid_to_cid[labels_list[image_id-1]]

  entry = {
      'file_name': tf.constant(image_name),
      'is_problematic': tf.constant(is_problematic),
      'correct_multi_labels': correct_cids,
      'wrong_multi_labels': wrong_cids,
      'unclear_multi_labels': unclear_cids,
      'label': original_label,
      'original_label': tf.constant(original_label),
  }
  imagenet_multilabel_v3[image_name] = entry

  if image_name in imagenet_m_2022_files:
    imagenet_m_2022[image_name] = entry


# Full imagenet validation labels.
imagenet_val = {}
for i, label in enumerate(labels_list):
  index = i + 1
  file_name = f'ILSVRC2012_val_{index:08}.JPEG'
  imagenet_val[file_name] = {
      'file_name': tf.constant(file_name),
      'label': wnid_to_cid[label],
  }

# Load vit3b and greedysoups logits into dictionaries.
vit3b = load_npy_as_dict('vit3b.npz')
greedysoups = load_npy_as_dict('greedysoups.npz')

In [ ]:
# Evaluate on ImageNet-M

# We expect ViT3B to get 0 on ImageNet-M by construction.
print('ImageNet-M ViT3B: ', imagenet_m_correct(imagenet_m_2022.values(), vit3b))

# Greedy Soups gets 19 correct.
print('ImageNet-M greedysoups: ', imagenet_m_correct(imagenet_m_2022.values(), greedysoups))

In [ ]:
# Evaluate ImageNet Multi-Label Accuracy
print('ImageNet-MLA ViT3B: ', imagenet_multilabel_accuracy(imagenet_multilabel_v3.values(), vit3b))
print('ImageNet-MLA greedysoups: ', imagenet_multilabel_accuracy(imagenet_multilabel_v3.values(), greedysoups))

In [ ]:
# Evaluate ImageNet Top-1 Accuracy
print('ImageNet-Top-1 ViT3B: ', imagenet_val_accuracy(imagenet_val.values(), vit3b))
print('ImageNet-Top-1 greedysoups: ', imagenet_val_accuracy(imagenet_val.values(), greedysoups))